# 2.2 扩展答疑机器人的知识范围

## 🚄 前言

你已经了解到RAG是扩展大模型知识范围的有效解决方案。本节你将学习RAG的工作流程，以及如何创建一个RAG应用，使它能够根据公司的制度文件来进行回答。

## 🍁 课程目标

学完本课程后，你将能够：<br>
1. 了解RAG的工作流程
2. 创建一个RAG应用

## 1. RAG的工作原理

你在考试的时候有可能会因为忘记某个概念或公式而失去分数，但考试如果是开卷形式，那么你只需要找到与考题最相关的知识点，并加上你的理解就可以进行回答了。

对于大模型来说也是如此，在训练过程中由于没有见过某个知识点（比如你们公司的制度文件），因此直接向它提问相关问题会得到不准确的答案；如果在大模型生成内容时，像开卷考试一样将相关知识提供给它作为参考，那么大模型回答的质量也就会大幅提高了。

RAG（Retrieval Augmented Generation）就是为大模型提供参考资料的解决方案。RAG应用通常包含**建立索引**与**检索生成**两部分。

### 1.1 建立索引
你可能会在考试前对参考资料做标记，来帮助你在考试时更容易地找到相关信息。类似的，RAG应用往往也会在回答前就已经做好了标记，这一过程叫做**建立索引**，建立索引包括四个步骤：<br>
1. **文档解析**<br>
就像你会将书上看到的视觉信息理解为文字信息一样，RAG应用也需要首先将知识库文档进行加载并解析为大模型能够理解的文字形式。
2. **文本分段**<br>
你通常不会在做某道题时把整本书都翻阅一遍，而是去查找与问题最相关的几个段落，因此你会先把参考资料做一个大致的分段。类似的，RAG应用也会在文档解析后对文本进行分段，以便于在后续能够快速找到与提问最相关的内容。
3. **文本向量化**<br>
在开卷考试时，你通常会先在参考资料中寻找与问题最相关的段落，再去进行作答。在RAG应用中，通常需要借助嵌入（embedding）模型分别对段落与问题进行数字化表示，在进行相似度比较后找出最相关的段落，数字化表示的过程就叫做文本向量化。<br>
    > 如果你对这一过程的细节感兴趣，可以学习-本教程的拓展阅读部分。
4. **存储索引**<br>
存储索引将向量化后的段落存储为向量数据库，这样RAG应用就无需在每次进行回复时都重复以上步骤，从而可以增加响应速度。

    <img src="https://gw.alicdn.com/imgextra/i2/O1CN010zLf411zVoZQ9cWsI_!!6000000006720-2-tps-1592-503.png" width="600"><br>

    在建立索引后，RAG应用就可以根据用户的问题检索出相关的文本段了。

### 1.2 检索生成
检索、生成分别对应着RAG名字中的`Retrieval`与`Generation`两阶段。**检索**就像开卷考试时去查找资料的过程，**生成**则是在找到资料后，根据参考资料与问题进行作答的过程。<br>
1. **检索**<br>
检索阶段会召回与问题最相关的文本段。通过embedding模型对问题进行文本向量化，并与向量数据库的段落进行语义相似度的比较，找出最相关的段落。检索是RAG应用中最重要的环节，你可以想象如果考试的时候找到了错误的资料，那么回答一定是不准确的。为了提高检索准确性，除了使用性能强大的embedding模型，也可以做重排（rerank）、句子窗口检索等方法，这些内容你可以在下一章节进行学习。
2. **生成**<br>
在检索到相关的文本段后，RAG应用会将问题与文本段通过提示词模板生成最终的提示词，由大模型生成回复，这个阶段更多是利用大模型的总结能力，而不是大模型本身具有的知识。
    > 一个典型的提示词模板为：`请根据以下信息回答用户的问题：{召回文本段}。用户的问题是：{question}。`

    <img src="https://img.alicdn.com/imgextra/i1/O1CN01vbkBXC1HQ0SBrC1Ii_!!6000000000751-2-tps-1776-639.png" width="600"><br>


<!-- 
它的整体流程图为：
<img src="https://img.alicdn.com/imgextra/i1/O1CN01GjUPUs1dIbaxdDowf_!!6000000003713-2-tps-1971-1221.png" width="600" />
1. **文档解析**<br>
就像你将书上看到的信息理解为文字信息一样，RAG应用也需要首先将知识库文档进行加载并解析为大模型能够理解的文字形式；
2. **文本分段**<br>
你通常不会在做某道题时把整本书都翻阅一遍，而是查找与问题最相关的几个段落，分段方法可能是按照目录中的标题等方法，RAG应用也会在解析后将文本分段，以便于后续的检索。
3. **索引建立**<br>
在开卷考试时，通常会先找到与问题最相关的几个段落，然后根据这些段落进行回答。RAG应用也会使用嵌入模型（embedding 模型）对切片后的文本段建立索引，并以向量数据库形式存储，便于后续检索。
使用嵌入模型（embedding 模型）对切片后的文档建立索引，并以向量数据库形式存储，便于后续检索；
4. 发起提问：用户输入的问题与向量数据库的知识进行匹配，并通过大模型生成结合知识库与用户问题的回复。 -->

## 2. 创建RAG应用

构建一个RAG应用需要实现以上功能，这个过程并不容易。但通过LlamaIndex，你不需要过多代码就可以完成上述功能。

与上一节教程一样，你需要运行以下代码将百炼API Key配置到环境中。

In [ ]:
from config.load_key import load_key
import os

load_key()
# 生产环境中请勿将 API Key 输出到日志中，避免泄露
print(f'''你配置的 API Key 是：{os.environ["DASHSCOPE_API_KEY"][:5]+"*"*5}''')

我们在docs文件夹中准备了一些虚构的公司制度文件，接下来你将基于这些文件来创建RAG应用。

In [ ]:
# 导入依赖
from llama_index.embeddings.dashscope import DashScopeEmbedding,DashScopeTextEmbeddingModels
from llama_index.core import SimpleDirectoryReader,VectorStoreIndex
from llama_index.llms.openai_like import OpenAILike

# 这两行代码是用于消除 WARNING 警告信息，避免干扰阅读学习，生产环境中建议根据需要来设置日志级别
import logging
logging.basicConfig(level=logging.ERROR)

print("正在解析文件...")
# LlamaIndex提供了SimpleDirectoryReader方法，可以直接将指定文件夹中的文件加载为document对象，对应着解析过程
documents = SimpleDirectoryReader('./docs').load_data()

print("正在创建索引...")
# from_documents方法包含切片与建立索引步骤
index = VectorStoreIndex.from_documents(
    documents,
    # 指定embedding 模型
    embed_model=DashScopeEmbedding(
        # 你也可以使用阿里云提供的其它embedding模型：https://help.aliyun.com/zh/model-studio/getting-started/models#3383780daf8hw
        model_name=DashScopeTextEmbeddingModels.TEXT_EMBEDDING_V2
    ))
print("正在创建提问引擎...")
query_engine = index.as_query_engine(
    # 设置为流式输出
    streaming=True,
    # 此处使用qwen-plus模型，你也可以使用阿里云提供的其它qwen的文本生成模型：https://help.aliyun.com/zh/model-studio/getting-started/models#9f8890ce29g5u
    llm=OpenAILike(
        model="qwen-plus",
        api_base="https://dashscope.aliyuncs.com/compatible-mode/v1",
        api_key=os.getenv("DASHSCOPE_API_KEY"),
        is_chat_model=True
        ))
print("正在生成回复...")
streaming_response = query_engine.query('我们公司项目管理应该用什么工具')
print("回答是：")
# 采用流式输出
streaming_response.print_response_stream()

### 2.1 保存与加载索引
你可能会发现，创建索引消耗的时间比较长。如果能够将索引保存到本地，并在需要使用的时候直接加载，而不是重新建立索引，那就可以大幅提升回复的速度，LlamaIndex提供了简单易实现的保存与加载索引的方法。

In [ ]:
# 将索引保存为本地文件
index.storage_context.persist("knowledge_base/test")
print("索引文件保存到了knowledge_base/test")

In [ ]:
# 将本地索引文件加载为索引
from llama_index.core import StorageContext,load_index_from_storage
storage_context = StorageContext.from_defaults(persist_dir="knowledge_base/test")
index = load_index_from_storage(storage_context,embed_model=DashScopeEmbedding(
        model_name=DashScopeTextEmbeddingModels.TEXT_EMBEDDING_V2
    ))
print("成功从knowledge_base/test路径加载索引")

从本地加载索引后，你可以再次进行提问测试是否可以正常工作。

In [ ]:
print("正在创建提问引擎...")
query_engine = index.as_query_engine(
    # 设置为流式输出
    streaming=True,
    # 此处使用qwen-plus模型，你也可以使用阿里云提供的其它qwen的文本生成模型：https://help.aliyun.com/zh/model-studio/getting-started/models#9f8890ce29g5u
    llm=OpenAILike(
        model="qwen-plus",
        api_base="https://dashscope.aliyuncs.com/compatible-mode/v1",
        api_key=os.getenv("DASHSCOPE_API_KEY"),
        is_chat_model=True
        ))
print("正在生成回复...")
streaming_response = query_engine.query('我们公司项目管理应该用什么工具')
print("回答是：")
streaming_response.print_response_stream()

你可以将上述代码进行封装，以便在后续持续迭代时快速使用。

In [ ]:
from chatbot import rag

# 引文在前面的步骤中已经建立了索引，因此这里可以直接加载索引。如果需要重建索引，可以增加一行代码：rag.indexing()
index = rag.load_index(persist_path='./knowledge_base/test')
query_engine = rag.create_query_engine(index=index)

rag.ask('我们公司项目管理应该用什么工具', query_engine=query_engine)

### 2.2 多轮对话
RAG的多轮对话与直接向大模型发起多轮对话机制略有不同。你已经从2.1的教程中了解到多轮对话可以让大模型参考历史对话信息，实现方法为将历史对话信息添加到messages列表中。

在RAG应用中的检索阶段，通常会去比较用户输入与文本段的语义相似度，但直接将用户输入与文本段进行相似度比较，可能会丢失历史对话信息，造成检索结果不准确。

假设用户在第一轮对话提问：”张三工位在哪里？“后，在第二轮对话提问：”他的主管是谁？“，如果直接将第二轮对话中的提问与文本段进行相似度比较，检索系统并不知道“他”指代的是谁，因此大概率会检索出错误的文本段。

如果将完整历史对话与问题都输入到检索系统，由于字数较多，检索系统可能无法处理（embedding模型在长文本上效果差于短文本）。业界常用的解决方法是：

1. 通过大模型，基于历史对话信息，将用户的问题改写为一个新的query，新的query将包含历史对话的关键信息。
2. 使用新的query，按照原先流程进行检索与生成的过程。

LlamaIndex提供了便捷的工具，可以快速实现RAG应用的多轮对话。

In [ ]:
from llama_index.core import PromptTemplate
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core.chat_engine import CondenseQuestionChatEngine

custom_prompt = PromptTemplate(
    """\
给定一段对话（在人类与助手之间）以及来自人类的后续信息，
请将该信息改写为一个独立的问题，并在其中包含对话中的所有相关上下文。

<聊天记录>
{chat_history}

<后续信息>
{question}

<独立的问题>
"""
)

# 历史对话信息
custom_chat_history = [
    ChatMessage(role=MessageRole.USER,content="内容开发工程师细分类型是什么？",),
    ChatMessage(role=MessageRole.ASSISTANT, content="综合技术岗位。"),
]

query_engine = index.as_query_engine(
    # 设置为流式输出
    streaming=True,
    # 此处使用qwen-plus模型，你也可以使用阿里云提供的其它qwen的文本生成模型：https://help.aliyun.com/zh/model-studio/getting-started/models#9f8890ce29g5u
    llm=OpenAILike(
        model="qwen-plus",
        api_base="https://dashscope.aliyuncs.com/compatible-mode/v1",
        api_key=os.getenv("DASHSCOPE_API_KEY"),
        is_chat_model=True
        ))
chat_engine = CondenseQuestionChatEngine.from_defaults(
    query_engine=query_engine,
    condense_question_prompt=custom_prompt,
    chat_history=custom_chat_history,
    llm=OpenAILike(
        model="qwen-plus",
        api_base="https://dashscope.aliyuncs.com/compatible-mode/v1",
        api_key=os.getenv("DASHSCOPE_API_KEY"),
        is_chat_model=True
        ),
    verbose=True
)

streaming_response = chat_engine.stream_chat("核心职责是什么")
for token in streaming_response.response_gen:
    print(token, end="")


虽然最后一个问题没有提到“内容开发工程师”，但大模型还是根据历史对话信息，将问题改写为“内容开发工程师的核心职责是什么？”，并给出了正确的答案。

## 📝3.本节小结
通过本节教程，你学习到了RAG的工作流程以及创建方法。

在RAG系统中，最重要的环节是**检索**，因为"garbage in, garbage out"，你如果喂给大模型错误的参考信息，那么大模型生成的回复大概率是不准确的。

如果你的RAG应用性能遇到了瓶颈（性能评估可以学习后续课程：自动化评测答疑机器人的表现），你可以将优化方向先聚焦在检索阶段上，比如对embedding模型进行微调、优化文本分段策略、修改召回文本段个数、设置相似度阈值、使用rerank模型增加排序精度等。当检索阶段不再带来性能上的提升时，你可以再去考虑在生成阶段进行优化，比如使用性能更强大的大模型、优化提示词模板等。

在下一节内容中，你将学习如何通过优化提示词来改善答疑机器人的回复质量。

### 拓展阅读

#### 文本向量化
计算机并不能直接理解“我喜欢吃苹果”与“我爱吃苹果”这两句话到底有多相似，但它能理解两个相同维度的向量的相似度（通常使用余弦相似度来衡量）。文本向量化通过embedding模型，将自然语言转化为计算机能够理解的数字形式。

embedding模型的训练通常会包含**对比学习**的环节，输入数据是许多已标记为是否相关的文本对(s1,s2)，模型的训练目标是尽可能让相关的文本对生成的向量相似度变高，不相关的文本对生成的向量相似度变低。

在**建立索引**阶段，假设已经通过文本分段获得了n个chunk：[c1,c2,c3,...,cn]，那么embedding模型会将这n个chunk分别转化为向量：[v1,v2,v3,...,vn]，并存储为向量数据库。

在**检索**阶段，假设用户的问题为q，那么embedding模型会将问题q转化为向量vq，并在向量数据库中找出与vq最相似的n个向量（这个值你可以自己设定），通过向量与文本段的索引关系得到文本段，作为检索结果。


## 课后小测验
【单选题】 在RAG应用中进行多轮对话，应该如何进行检索？

A. 在检索阶段输入完整的历史对话信息<br>
B. 结合历史对话信息对输入问题改写后进入检索阶段<br>
C. 在检索阶段输入最新的输入问题<br>
D. 将上一轮召回的文本段迁移过来<br>
参考答案：B<br>

## ✅评价反馈
欢迎你参与[阿里云大模型ACP课程问卷](https://survey.aliyun.com/apps/zhiliao/Mo5O9vuie) 反馈学习体验和课程评价。
你的批评和鼓励都是我们前进的动力！